In [1]:
import pandas as pd
import numpy as np
from scipy import stats

[For starting kaggle](https://towardsdatascience.com/how-i-got-started-with-kaggle-competitions-its-not-that-hard-7666f25e9f93)

## Hypothesis testing

When we do hypothesis testing, we actually have two statements of interest: the first is our actual
explanation, which we call the alternative hypothesis, and the second is that the explanation we have is not
sufficient, and we call this the null hypothesis. Our actual testing method is to determine whether the null
hypothesis is true or not. If we find that there is a difference between groups, then we can reject the null
hypothesis and we accept our alternative.

In [2]:
df = pd.read_csv('datasets/grades.csv')
df.head()

,student_id,assignment1_grade,assignment1_submission,assignment2_grade,assignment2_submission,assignment3_grade,assignment3_submission,assignment4_grade,assignment4_submission,assignment5_grade,assignment5_submission,assignment6_grade,assignment6_submission
0,B73F2C11-70F0-E37D-8B10-1D20AFED50B1,92.733946,2015-11-02 06:55:34.282000000,83.030552,2015-11-09 02:22:58.938000000,67.164441,2015-11-12 08:58:33.998000000,53.011553,2015-11-16 01:21:24.663000000,47.710398,2015-11-20 13:24:59.692000000,38.168318,2015-11-22 18:31:15.934000000
1,98A0FAE0-A19A-13D2-4BB5-CFBFD94031D1,86.790821,2015-11-29 14:57:44.429000000,86.290821,2015-12-06 17:41:18.449000000,69.772657,2015-12-10 08:54:55.904000000,55.098125,2015-12-13 17:32:30.941000000,49.588313,2015-12-19 23:26:39.285000000,44.629482,2015-12-21 17:07:24.275000000
2,D0F62040-CEB0-904C-F563-2F8620916C4E,85.512541,2016-01-09 05:36:02.389000000,85.512541,2016-01-09 06:39:44.416000000,68.410033,2016-01-15 20:22:45.882000000,54.728026,2016-01-11 12:41:50.749000000,49.255224,2016-01-11 17:31:12.489000000,44.329701,2016-01-17 16:24:42.765000000
3,FFDF2B2C-F514-EF7F-6538-A6A53518E9DC,86.030665,2016-04-30 06:50:39.801000000,68.824532,2016-04-30 17:20:38.727000000,61.942079,2016-05-12 07:47:16.326000000,49.553663,2016-05-07 16:09:20.485000000,49.553663,2016-05-24 12:51:18.016000000,44.598297,2016-05-26 08:09:12.058000000
4,5ECBEEB6-F1CE-80AE-3164-E45E99473FB4,64.813800,2015-12-13 17:06:10.750000000,51.491040,2015-12-14 12:25:12.056000000,41.932832,2015-12-29 14:25:22.594000000,36.929549,2015-12-28 01:29:55.901000000,33.236594,2015-12-29 14:46:06.628000000,33.236594,2016-01-05 01:06:59.546000000


In [6]:
df.shape
df.dtypes

student_id                 object
assignment1_grade         float64
assignment1_submission     object
assignment2_grade         float64
assignment2_submission     object
assignment3_grade         float64
assignment3_submission     object
assignment4_grade         float64
assignment4_submission     object
assignment5_grade         float64
assignment5_submission     object
assignment6_grade         float64
assignment6_submission     object
dtype: object

In [7]:
pd.to_datetime(df['assignment1_submission']).dtype

dtype('<M8[ns]')

In [42]:
#method 1
early_finisher = df[df['assignment1_submission'] < '2016']
late_finisher = df[df['assignment1_submission'] >= '2016']

In [33]:
#method2
df2 = pd.read_csv('datasets/grades.csv')
df2 = df2.set_index([df2.index, 'assignment1_submission'])
def segment(item):
    if item[1] >= '2016':
        return 'late finisher'
    elif item[1] < '2016':
        return 'early finisher'
finisher = df2.groupby(segment)
for key, group in finisher:
    print(key, group)
df2 = df2.reset_index()

early finisher                                                               student_id  \
     assignment1_submission                                                
0    2015-11-02 06:55:34.282000000  B73F2C11-70F0-E37D-8B10-1D20AFED50B1   
1    2015-11-29 14:57:44.429000000  98A0FAE0-A19A-13D2-4BB5-CFBFD94031D1   
4    2015-12-13 17:06:10.750000000  5ECBEEB6-F1CE-80AE-3164-E45E99473FB4   
5    2015-12-28 04:35:32.836000000  D09000A0-827B-C0FF-3433-BF8FF286E15B   
8    2015-12-25 02:29:28.415000000  C9D51293-BD58-F113-4167-A7C0BAFCB6E5   
...                                                                  ...   
2308 2015-10-03 09:04:46.358000000  EFDA9F93-D0C3-864F-B0F6-2E9AA3E05E31   
2309 2015-10-01 23:25:20.529000000  6D2AB78F-44F4-2E8B-5C5E-B79119BC7EAC   
2311 2015-11-29 02:43:27.932000000  DE88902E-C7A7-E37A-CFA7-F2C8F2D219F2   
2312 2015-10-20 08:09:27.418000000  EFDA9F93-D0C3-864F-B0F6-2E9AA3E05E31   
2314 2015-10-04 09:46:03.403000000  DDE0526B-7DA4-80E8-C2A6-D097F3826029 

In [40]:
#method 3
late_finisher = df.merge(early_finisher,how='left')
late_finisher.shape

(2315, 13)

In [ ]:
#method 4
late_finisher = df[~df.index.isin(early_finisher.index)] #좋은 생각인 것 같음. Reverse bitwise

In [44]:
df.mean() #We can see only columns whose dtype is number are calculated

assignment1_grade    74.535732
assignment2_grade    66.849007
assignment3_grade    60.623197
assignment4_grade    54.112112
assignment5_grade    48.618522
assignment6_grade    43.841452
dtype: float64

When doing hypothesis testing, we have to choose a **significance level** as a threshold for how much of a
chance we're willing to accept. This significance level is typically called alpha. #For this example, let's
use a threshold of 0.05 for our alpha or 5%. Now this is a commonly used number but it's really quite
arbitrary.

The SciPy library contains a number of different statistical tests and forms a basis for hypothesis testing
in Python and we're going to use the ttest_ind() function which does an independent t-test (meaning the
populations are not related to one another). The result of ttest_index() are the t-statistic and a p-value.
It's this latter value, the probability, which is most important to us, as it indicates the chance (between
0 and 1) of our null hypothesis being True.

[Significant level & p value](https://blog.minitab.com/en/adventures-in-statistics-2/understanding-hypothesis-tests-significance-levels-alpha-and-p-values-in-statistics)

[T test](https://bioinformaticsandme.tistory.com/80)

In [51]:
stats.ttest_ind(early_finisher['assignment1_grade'],late_finisher['assignment1_grade'])

Ttest_indResult(statistic=1.322354085372139, pvalue=0.1861810110171455)

So here we see that the probability is 0.18, and this is above our alpha value of 0.05. **This means that we
cannot reject the null hypothesis.** The null hypothesis was that the two populations are the same, and we
don't have enough certainty in our evidence (because it is greater than alpha) to come to a conclusion to
the contrary. This doesn't mean that we have proven the populations are the same.

In [57]:
print(stats.ttest_ind(early_finisher['assignment1_grade'],late_finisher['assignment1_grade']))
print(stats.ttest_ind(early_finisher['assignment2_grade'],late_finisher['assignment2_grade']))
print(stats.ttest_ind(early_finisher['assignment3_grade'],late_finisher['assignment3_grade']))
print(stats.ttest_ind(early_finisher['assignment4_grade'],late_finisher['assignment4_grade']))
print(stats.ttest_ind(early_finisher['assignment5_grade'],late_finisher['assignment5_grade']))
print(stats.ttest_ind(early_finisher['assignment6_grade'],late_finisher['assignment6_grade']))

Ttest_indResult(statistic=1.322354085372139, pvalue=0.1861810110171455)
Ttest_indResult(statistic=1.2514717608216366, pvalue=0.2108889627004424)
Ttest_indResult(statistic=1.6133726558705392, pvalue=0.10679998102227865)
Ttest_indResult(statistic=0.049671157386456125, pvalue=0.960388729789337)
Ttest_indResult(statistic=-0.05279315545404755, pvalue=0.9579012739746492)
Ttest_indResult(statistic=-0.11609743352612056, pvalue=0.9075854011989656)


Ok, so it looks like in this data we do not have enough evidence to suggest the populations differ with
respect to grade. Let's take a look at those p-values for a moment though, because they are saying things
that can inform experimental design down the road. For instance, one of the assignments, assignment 3, has a
p-value around 0.1. This means that if we accepted a level of chance similarity of 11% this would have been
considered statistically significant. As a research, this would suggest to me that there is something here
worth considering following up on. For instance, if we had a small number of participants (we don't) or if
there was something unique about this assignment as it relates to our experiment (whatever it was) then
there may be followup experiments we could run.

P-values have come under fire recently for being insufficient for telling us enough about the interactions
which are happening, and two other techniques, **confidence intervalues and bayesian analyses, are being used
more regularly**. **One issue with p-values is that as you run more tests you are likely to get a value which
is statistically significant just by chance.**

In [72]:
df_ex = pd.DataFrame(np.random.rand(100,100))
df_ex2 = pd.DataFrame(np.random.rand(100,100))
stats.ttest_ind(df_ex,df_ex2)

Ttest_indResult(statistic=array([ 0.74440233,  1.95016934,  0.85903557, -1.04954102,  0.36512299,
        1.35406883,  0.59897937, -0.22080219, -0.55862255, -0.818488  ,
        0.07949126,  0.74448219, -0.26022535, -1.51028577,  0.61890436,
        0.35370372, -1.10268603, -1.83206296, -0.03939916,  0.70897945,
       -0.12080411, -0.07231679, -0.31560911,  0.43594949,  0.19921982,
       -0.49613938,  0.1129025 ,  0.01019079,  0.48434737,  0.49612944,
        0.23430764, -0.26176824, -0.26256872, -0.98984275, -0.3249166 ,
       -0.58176275, -0.16777779, -1.09051278,  0.08547424, -0.0417833 ,
        0.58118745,  1.49373407, -1.30964263, -0.54241183,  0.65569986,
       -1.23428727, -0.20861231,  0.42640594,  1.80517292, -0.20082946,
        0.8704161 , -1.98386892,  1.85228758,  0.78257032,  2.05661069,
       -0.19494403, -0.93533229,  0.18663802, -1.49777005,  0.29231155,
       -0.06490491,  0.02399493, -1.45676909, -0.13595557, -0.16090491,
        1.96290999,  1.15551915,  0.18

In [78]:
df_ex = pd.DataFrame(np.random.rand(100,100))
df_ex2 = pd.DataFrame(np.random.rand(100,100))
def test_columns(df1, df2, alpha=0.1):
    result_set= {}
    for col in df1.columns:
        teststat, pval = stats.ttest_ind(df1[col],df2[col])
        if pval <= alpha:
            result_set[col] = (alpha,pval)
    return result_set
test_columns(df_ex,df_ex2,0.20)

{1: (0.2, 0.11627172396837271),
 4: (0.2, 0.18182997093847114),
 23: (0.2, 0.02487787254453243),
 45: (0.2, 0.015384106950749963),
 49: (0.2, 0.19309249075530124),
 50: (0.2, 0.05914284464746164),
 51: (0.2, 0.1089637612271509),
 56: (0.2, 0.13210225814278265),
 63: (0.2, 0.08040555344413186),
 68: (0.2, 0.16501476691728673),
 69: (0.2, 0.144232086383631),
 71: (0.2, 0.005862764641698116),
 75: (0.2, 0.10694163203454717),
 83: (0.2, 0.15516197042491142),
 87: (0.2, 0.1476348045652125),
 89: (0.2, 0.1791555536675419),
 90: (0.2, 0.006636490592130091),
 94: (0.2, 0.011073037273612422),
 95: (0.2, 0.19572701916314267)}

Interesting, so we see that there are a bunch of columns that are different! In fact, that number looks a
lot like the alpha value we chose. So what's going on - shouldn't all of the columns be the same? Remember
that all the ttest does is check if two sets are similar given some level of confidence, in our case, 10%.
The more random comparisons you do, **the more will just happen to be the same by chance.** In this example, we
checked 100 columns, so we would expect there to be roughly 10 of them if our alpha was 0.1.

In [80]:
df_ex2=pd.DataFrame([np.random.chisquare(df=1,size=100) for x in range(100)])
test_columns(df_ex,df_ex2)

{0: (0.1, 0.0013449765735121491),
 1: (0.1, 3.394911401877077e-06),
 2: (0.1, 0.007445087334119664),
 3: (0.1, 0.0006386180956930828),
 4: (0.1, 0.00014219746397301514),
 5: (0.1, 1.776541826206327e-05),
 6: (0.1, 0.0003015246215136257),
 7: (0.1, 0.00010604005413461184),
 8: (0.1, 0.00035549796671048654),
 9: (0.1, 0.0005608797807600568),
 10: (0.1, 0.006085694655409246),
 11: (0.1, 0.0001867671801334664),
 12: (0.1, 3.762366547589457e-05),
 13: (0.1, 0.00030505318971975706),
 14: (0.1, 0.00046281997354480044),
 15: (0.1, 0.0006144521016156738),
 16: (0.1, 0.008688563986004597),
 17: (0.1, 0.001064249906910045),
 18: (0.1, 0.0016219897366607037),
 19: (0.1, 0.0002216748495119436),
 20: (0.1, 0.00247421934915673),
 21: (0.1, 7.089798816575526e-06),
 22: (0.1, 0.00043205316670595335),
 23: (0.1, 0.00025130783067709323),
 24: (0.1, 3.266009763127041e-07),
 25: (0.1, 3.3773608312286316e-05),
 26: (0.1, 0.0027829823741488823),
 27: (0.1, 0.0043923233413999474),
 28: (0.1, 0.000235413906668

## Structured data

It's an excellent way to show relationships between instances which are sometimes called observations, entities, or in the case of Pandas, rows, and the attributes that these instances have. Which are often called features or in Pandas, columns. But it's important to recognize that there's nothing in it to the data which is structured this way. This tabular representation of data is an abstraction that we as data scientists have applied to allow us to more easily build manipulation routines with certain properties. It's an abstraction that allows us to quickly do operations such as summarizing data fields or applying a machine learning algorithm. But there are many different forms of abstractions that you should be aware of, and I want to touch on a few of those here.



### Network Diagram - Social Research

### Adjacency Matrix

### Tree(Hierarchical Graph) - NLP

These structures are representations that we impose on the underlying data. The meaning of the data that we derive from these representations can change based on how we decide to apply a representation. A data scientist needs to be able to interact with a broad array of other stakeholders and as such needs to be able to be flexible with how they conceive of and represent data.